# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Вывод.<br> 1.Данные не имеют отсутствующих значений.<br> 2.Числовые данные (количественные значения минут звонков, количества смс, использованного интернет трафика) представлены в формате float. Данные в целевом слобце is_ultra представлены в формате int.

## Разбейте данные на выборки

In [4]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.2, random_state=12345)

In [5]:
features_train, features_test, target_train, target_test = train_test_split(features_train, target_train, 
                                                                            test_size=0.25, random_state=12345)

Исходная выборка разделена в пропорциях 3 (обучающая выборка), 1 (валидационная выборка), 1 (тестовая выборка)

## Исследуйте модели

### Дерево решений

In [6]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
print('Accuracy лучшей модели', best_result, 'Лучшая глубина дерева', best_depth)

Accuracy лучшей модели 0.7884914463452566 Лучшая глубина дерева 5


### Случайный лес

In [7]:
best_model_forest = None
best_result_forest = 0
best_est = 0
for est in range(1,10):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train)
    predictions_valid_forest = model_forest.predict(features_valid)
    result_forest = accuracy_score(target_valid, predictions_valid_forest)
    if result_forest > best_result_forest:
        best_model_forest = model_forest
        best_result_forest = result_forest
        best_est = est
    
        
print('Accuracy лучшей модели', best_result_forest)

Accuracy лучшей модели 0.7807153965785381


### Логистическая регрессия

In [8]:
best_model_regr = None
best_result_regr = 0
best_iter = 0
for itera in range(500, 5001, 500):
    model_regr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=itera)
    model_regr.fit(features_train, target_train)
    predictions_valid_regr = model_regr.predict(features_valid)
    result_regr = model_regr.score(features_valid, target_valid)
    if result_regr>best_result_regr:
        best_model_regr=model_regr
        best_result_regr = result_regr
        best_iter=itera
print('Accuracy модели логистической регрессии',best_result_regr)

Accuracy модели логистической регрессии 0.7589424572317263


Сформированы и обучены и проверены на валидационных данных разные алгоритмы предсказаний. На используемыех данных с большей точностью предсказания осуществлены алгоритмом "Дерево решений".

### GreadsearchCV

подберем параметры моделей с помощью функции GridSearchCV и опробуем их на валидационной выборке

#### Случайный лес

In [9]:
rf = RandomForestClassifier()
params_rf = {'n_estimators': range(10, 100, 10),
             'max_depth': range(1, 16, 1),
             'min_samples_split': range(1, 11, 2),
             'min_samples_leaf': range(1, 11, 2)}
grid_rf = GridSearchCV(rf, params_rf, cv=5)
grid_rf.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 16),
                         'min_samples_leaf': range(1, 11, 2),
                         'min_samples_split': range(1, 11, 2),
                         'n_estimators': range(10, 100, 10)})

In [10]:
grid_rf.best_params_

{'max_depth': 13,
 'min_samples_leaf': 7,
 'min_samples_split': 3,
 'n_estimators': 30}

In [11]:
grid_rf.best_score_

0.8236552048987281

In [12]:
predict_grid_rf_valid = grid_rf.predict(features_valid)
predict_grid_rf_valid_score = accuracy_score(target_valid, predict_grid_rf_valid)
predict_grid_rf_valid_score

0.7962674961119751

In [13]:
predict_grid_rf_valid

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,

#### Дерево принятия решений

In [14]:
dt = DecisionTreeClassifier()
params_dt = {'max_depth': range(1, 10, 1),
             'min_samples_split': range(1, 11, 2),
             'min_samples_leaf': range(1, 11, 2)}
grid_dt = GridSearchCV(dt, params_dt, cv=5)
grid_dt.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 10),
                         'min_samples_leaf': range(1, 11, 2),
                         'min_samples_split': range(1, 11, 2)})

In [15]:
grid_dt.best_params_

{'max_depth': 8, 'min_samples_leaf': 9, 'min_samples_split': 3}

In [16]:
grid_dt.best_score_

0.8081030886212233

In [17]:
predict_grid_dt_valid = grid_dt.predict(features_valid)
predict_grid_dt_valid_score = accuracy_score(target_valid, predict_grid_dt_valid)
predict_grid_dt_valid_score

0.7853810264385692

#### Логистическая регрессия

In [18]:
lr=LogisticRegression()
params_lr = {'solver': ['newton-cg', 'lbfgs', 'sag', 'saga', 'liblinear'],
             'max_iter': range(100, 5001, 200)}
grid_lr = GridSearchCV(lr, params_lr, cv=5)
grid_lr.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'max_iter': range(100, 5001, 200),
                         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga',
                                    'liblinear']})

In [19]:
grid_lr.best_params_

{'max_iter': 100, 'solver': 'newton-cg'}

In [20]:
grid_lr.best_score_

0.7505201534217079

In [21]:
predict_grid_lr_valid = grid_lr.predict(features_valid)
predict_grid_lr_valid_score = accuracy_score(target_valid, predict_grid_lr_valid)
predict_grid_lr_valid_score

0.7589424572317263

Функция GridSearchCV помогла подобрать оптимальные параметры для моделей Случайный лес, Дерево принятия решения и Логистическая регрессия. Лучшие результаты accuracy на валидационной выборке показаны моделью Случайный лес.

## Проверьте модель на тестовой выборке

In [22]:
predictions_test_tree = best_model.predict(features_test)
result_test_tree = accuracy_score(target_test, predictions_test_tree)
print('Accuracy дерева решений', result_test_tree)

Accuracy дерева решений 0.7589424572317263


In [23]:
predictions_test_forest = best_model_forest.predict(features_test)
result_test_forest = accuracy_score(target_test, predictions_test_forest)
print('Accuracy случайного леса', result_test_forest)

Accuracy случайного леса 0.7620528771384136


In [24]:
result_test_regr = model_regr.score(features_test, target_test)
print('Accuracy логистической регрессии', result_test_regr)

Accuracy логистической регрессии 0.7262830482115086


In [25]:
predict_grid_rf_test = grid_rf.predict(features_test)
predict_grid_rf_test_score = accuracy_score(target_test, predict_grid_rf_test)
predict_grid_rf_test_score

0.7947122861586314

In [26]:
predict_grid_dt_test = grid_dt.predict(features_test)
predict_grid_dt_test_score = accuracy_score(target_test, predict_grid_dt_test)
predict_grid_dt_test_score

0.7791601866251944

In [27]:
predict_grid_lr_test = grid_lr.predict(features_test)
predict_grid_lr_test_score = accuracy_score(target_test, predict_grid_lr_test)
predict_grid_lr_test_score

0.7262830482115086

Вывод.<br> 1.Данные тестовой выборки обработаны моделями построенными на "ручном" подборе параметров и на автоматическом подборе параметров.<br> 2. Модели сформированные на автоматическом подборе параметров показали лучшие параметры accuracy чем модели с "ручным" подбором параметров. 3.Модель Случайного леса имеет большую точность передсказаний на тествой выборке чем модели Дерева принятия решения и Логистической регрессии (как при "ручном" подборе параметров, так и при автоматическом) <br> 4. Модель Случайного леса (grid_rf), сформированная автоматическим подбором параметров (функция GridSearchCV) предлагается рассмотреть далее для определения прочих параметров предсказаний.<br> 5. Из "ручных" моделей также большую accuracy на тестовой выборке показала модель Случайного леса. Данная модель так же предлагается рассмотреть для определения прочих параметров предсказаний.

## (бонус) Проверьте модели на адекватность

In [28]:
print(classification_report(target_test, best_model_forest.predict(features_test), 
                            target_names=['is_ultra', 'is_not_ultra']))

              precision    recall  f1-score   support

    is_ultra       0.80      0.86      0.83       443
is_not_ultra       0.64      0.54      0.58       200

    accuracy                           0.76       643
   macro avg       0.72      0.70      0.71       643
weighted avg       0.75      0.76      0.76       643



In [29]:
print(classification_report(target_test, grid_rf.predict(features_test), target_names=['is_ultra', 'is_not_ultra']))

              precision    recall  f1-score   support

    is_ultra       0.80      0.93      0.86       443
is_not_ultra       0.76      0.49      0.60       200

    accuracy                           0.79       643
   macro avg       0.78      0.71      0.73       643
weighted avg       0.79      0.79      0.78       643



Модель случайного леса (ручная)<br>
Точность (precision) модели составляет 80 %, что равно расчитанной на тестовой выборке Accuracy, то есть модель дает 80 % правильных ответов. Полнота (recall) модели составляет 86 %, то есть модель выделила 86 % пользователей тарифа Ultra. Модель с точностью 64 % предсказала пользователей у которых тарифный план не Ultra и выделила 54 % таких пользователей. <br>

Модель случайного леса (автоматическая)<br>
Точность (precision) модели составляет 79 %, что равно расчитанной на тестовой выборке Accuracy, то есть модель дает 76 % правильных ответов. Полнота (recall) модели составляет 94 %, то есть модель выделила 94 % пользователей тарифа Ultra. Точность модели в предсказании пользователей тарифа не Ultra составляет 76 %, модель выделила 45 % таких пользователей. <br>

Предлагается использовать Модель случайного леса(grid_rf) (автоматическую) для определения тарифа пользователей компании сотовой связи.


Выводы по проекту:<br>
1.Полученная выборка разделена на обучающую, валидационную и тестовую<br>
2.На обучающей выборке обучены модели дерева, случаного леса и логистической регрессии и выбраны лучшие модели данных типов.<br>
3.Лучшие модели по типам протестированы на тестовой выборке. Лучшее Accuracy получилось у моделей случайного леса.<br>
4.Для выбранных моделей случайного леса расчитаны показатели precision и recall, которые показали, что точность модели составляет 80 % и ее полнота 86 % (ручная модель) и 79% (pecision) 94 % (recall) у модели случайного леса подобранного функцией GridSearchCV.<br>
5.Для дальнейшей работы по предсказанию пользователей тарифа Ultra предлагается использовать модель случайного леса (автоматическую) - grid_rf
